In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//div[@class="span8"]/div[@class="row-fluid"]/h1/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                item_number = html.xpath('//div[@class="span8"]/div[@class="row-fluid"]/p[@class="product-details-code"]/text()')[0].replace('Item No:', '').strip()

                # = = = = = = = = = = = = = = =
                
                upc = html.xpath('//div[@class="span8"]/div[@class="row-fluid"]/p[@class="product-details-upccode"]/text()')
                upc = '' if len(upc) == 0 else upc[0].replace('UPC Code:', '').strip()
    
                # = = = = = = = = = = = = = = =
                                
                src = html.xpath('//img[@id="product-detail-gallery-main-img"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_li_1 = html.xpath('//li[@class="information-attribute"]')
                
                # = = = = = = = = = = = = = = =
                
                list_li_2 = html.xpath('//div[@class="row-fluid"]/div[@class="span6"]/ul[@class="unstyled"]/li/text()')
                list_li_2 = [li.strip() for li in list_li_2]

                while '' in list_li_2:
                    list_li_2.remove('')
                    
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Item_Number': item_number,
                                         'UPC': upc,
                                         'Title': title,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for li in list_li_1:
                    df_temp.loc[0, li.xpath('./span[@class="attribute-title"]/text()')[0].replace(':', '').strip().lower()] = li.xpath('./span[@class="attribute-value"]/text()')[0].strip()

                for li in list_li_2:
                    df_temp.loc[0, li.split(':')[0].strip().upper()] = li.split(':')[1].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：72

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/nylon-nuts-quik-select-kit-6807
[尝试次数：1] - [剩余数量：62] - [当前时间：17:03:12]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/nylon-rivet-quik-select-kit-6806
[尝试次数：1] - [剩余数量：61] - [当前时间：17:03:12]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/chry-xmas-tree-retainer-quik-select-kit-6803
[尝试次数：1] - [剩余数量：60] - [当前时间：17:03:13]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/ford-push-type-retainer-quik-select-kit-6805
[尝试次数：1] - [剩余数量：59] - [当前时间：17:03:13]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/gm-xmas-tree-retainer-quik-select-kit-6801
[尝试次数：1] - [剩余数量：58] - [当前时间：17:03:13]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/license-plate-hardware-quik-select-kit-6808
[尝试次数：1] - [剩余数量：57] - [当前时间：17:03:13]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/reveal

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/gm-reveal-moulding-clips-and-stud-screw-quik-select-kit-6876
[尝试次数：1] - [剩余数量：7] - [当前时间：17:03:40]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/spring-pin-quik-select-kit-6861
[尝试次数：1] - [剩余数量：6] - [当前时间：17:03:41]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/push-type-retainer-quick-select-kit-gm-ford-chry-6870
[尝试次数：1] - [剩余数量：5] - [当前时间：17:03:41]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/fender-washers-quick-select-kit-6869
[尝试次数：1] - [剩余数量：4] - [当前时间：17:03:41]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/retainer-quik-select-kit-6875
[尝试次数：1] - [剩余数量：3] - [当前时间：17:03:41]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/u-s-specialty-screw-quik-select-kit-6853
[尝试次数：1] - [剩余数量：2] - [当前时间：17:03:42]

[ok] - https://www.auveco.com/polypropylene-compartment-box-6800-series/nissan-